In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import os
import matplotlib.pyplot as plt

from math import sqrt
import datetime
import pickle

import sklearn
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn import linear_model

/Users/jjsm/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
%matplotlib inline

# Load data and past predictions

In [3]:
# import the pre-processed data
X = pickle.load(open('../data/fs_data_lags_1_12.pkl', 'rb'))

In [4]:
# predictions made by linear or xgb models, for month 33 (=validation) or month 34 (=test)
# models were also fitted with different lags, but otherwise with same parameters
fs_xgb_lags_1_12_preds_up_to_month_34 = pickle.load(open('fs_xgb_lags_1_12_preds_up_to_month_34.pkl', 'rb'))
fs_xgb_lags_1_12_preds_up_to_month_33 = pickle.load(open('fs_xgb_lags_1_12_preds_up_to_month_33.pkl', 'rb'))

fs_xgb_lags_1_to_6_preds_up_to_month_34 = pickle.load(open('fs_xgb_lags_1_to_6_preds_up_to_month_34.pkl', 'rb'))
fs_xgb_lags_1_to_6_preds_up_to_month_33 = pickle.load(open('fs_xgb_lags_1_to_6_preds_up_to_month_33.pkl', 'rb'))

fs_lm_lags_1_12_preds_up_to_month_33 = pickle.load(open('fs_lm_lags_1_12_preds_up_to_month_33.pkl', 'rb'))
fs_lm_lags_1_12_preds_up_to_month_34 = pickle.load(open('fs_lm_lags_1_12_preds_up_to_month_34.pkl', 'rb'))



# Fit a stacked model, using data up to month 33

In [5]:
# join the predictions for month 33 with a very restricted set of columns from the original data
month_33_predictions = pd.DataFrame({
    'fs_xgb_lags_1_12_preds'   : fs_xgb_lags_1_12_preds_up_to_month_33, 
    'fs_xgb_lags_1_to_6_preds' : fs_xgb_lags_1_to_6_preds_up_to_month_33,
    'fs_lm_lags_1_12_preds' : fs_lm_lags_1_12_preds_up_to_month_33
 })

month_33_X = X[['date_block_num', 'item_cnt_month']].query('date_block_num == 33')

month_33_data = pd.concat([month_33_X.reset_index(),
                           month_33_predictions], axis=1)

In [6]:
target_var = 'item_cnt_month'
predictor_cols = [x for x in month_33_data.columns if x not in (target_var, 'index', 'date_block_num')]

In [7]:
# define train and validation sets (note that we aren't splitting by month this time, since we'll be taking data
# for a single month and making predictions for that same month)
X_train, X_val, y_train, y_val = train_test_split(month_33_data[predictor_cols], 
                                                  month_33_data[target_var], 
                                                  test_size=0.33)

In [8]:
# xgboost doesn't like data frames
xgtrain = xgb.DMatrix(X_train.values,      
                      label=y_train.values,     
                      feature_names=predictor_cols)

xgval = xgb.DMatrix(X_val.values,      
                      label=y_val.values,     
                      feature_names=predictor_cols)


In [9]:
# other parameters
num_round = 5000
watchlist = [(xgtrain, 'train'),(xgval, 'eval')] # early stopping uses the last one listed here

In [10]:
params = {'alpha': 1,
          'colsample_bytree': 0.8,
          'eta': 0.1,
          'eval_metric': 'rmse',
          'gamma': 0,
          'max_depth': 8,
          'min_child_weight': 1,
          'objective': 'reg:linear',
          'silent': 0,
          'subsample': 0.8}



In [11]:
best_model = xgb.train(params, 
                  xgtrain,
                  num_round, 
                  watchlist, 
                  early_stopping_rounds=10)

[0]	train-rmse:1.32923	eval-rmse:1.41393
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 10 rounds.
[1]	train-rmse:1.27918	eval-rmse:1.36617
[2]	train-rmse:1.23258	eval-rmse:1.32555
[3]	train-rmse:1.20174	eval-rmse:1.30033
[4]	train-rmse:1.17723	eval-rmse:1.28059
[5]	train-rmse:1.14828	eval-rmse:1.25465
[6]	train-rmse:1.11991	eval-rmse:1.23264
[7]	train-rmse:1.09753	eval-rmse:1.21469
[8]	train-rmse:1.07463	eval-rmse:1.19565
[9]	train-rmse:1.05761	eval-rmse:1.18302
[10]	train-rmse:1.0421	eval-rmse:1.17182
[11]	train-rmse:1.03029	eval-rmse:1.16583
[12]	train-rmse:1.01543	eval-rmse:1.15397
[13]	train-rmse:1.00428	eval-rmse:1.14926
[14]	train-rmse:0.991277	eval-rmse:1.13829
[15]	train-rmse:0.980918	eval-rmse:1.13163
[16]	train-rmse:0.970989	eval-rmse:1.12723
[17]	train-rmse:0.96567	eval-rmse:1.12476
[18]	train-rmse:0.957328	eval-rmse:1.11881
[19]	train-rmse:0.952382	eval-rmse:1.11808
[20]	train-rmse:0.94509

In [12]:
# Evaluate
preds  = best_model.predict(xgval, ntree_limit=best_model.best_iteration)

In [13]:
# clip to 0-20 to match test data
preds = np.clip(preds, 0, 20)

In [14]:
# compare to the (similarly clipped) validation data
# sqrt(mean_squared_error(preds, X.iloc[validation_rows][target_var].values))
rmse = sqrt(mean_squared_error(preds, np.clip(y_val, 0, 20)))
rmse

0.8872311813646788

['fs_lm_lags_1_12_preds', 'fs_xgb_lags_1_12_preds', 'fs_xgb_lags_1_to_6_preds']

# Make predictions for the test data

In [15]:
# join the predictions for month 33 with a very restricted set of columns from the original data
month_34_predictions = pd.DataFrame({
    'fs_xgb_lags_1_12_preds'   : fs_xgb_lags_1_12_preds_up_to_month_34, 
    'fs_xgb_lags_1_to_6_preds' : fs_xgb_lags_1_to_6_preds_up_to_month_34,
    'fs_lm_lags_1_12_preds'    : fs_lm_lags_1_12_preds_up_to_month_34
 })

X_34 = X[['ID', 'date_block_num', 'item_category_id', 'item_cnt_month', 'item_id', 'shop_id']].query("date_block_num==34")

month_34_data = pd.concat([X_34.reset_index(),
                           month_34_predictions], axis=1)

In [16]:
preds  = best_model.predict(xgb.DMatrix(month_34_data[predictor_cols].values, feature_names=predictor_cols),
                            ntree_limit=best_model.best_iteration)
preds = np.clip(preds, 0, 20)


# Save the stacking model

In [17]:
pickle.dump(best_model, open('stacking_fs_xgb.pkl', 'wb'))

# Form submission file

In [46]:
# create submission file
submission_filename = 'future_sales_' + datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + '.csv'

submission = pd.DataFrame({
  'ID' : [int(x) for x in month_34_data.ID],
  'item_cnt_month' : preds
 })

# make sure this is in the same order as the test data
submission = submission.sort_values('ID')

In [49]:
submission.to_csv('../submissions/' + submission_filename,index=False)